In [1]:
luna_path = '/Volumes/solo/ali/'

annotations_path = luna_path + 'csv/val/'
src = luna_path + 'Data/val/'
dst_nodules = luna_path + 'Data/val_mask/'

In [2]:
import time
import gc
import os

import SimpleITK as sitk
import numpy as np
import csv
import scipy
from glob import glob
import pandas as pd
from scipy import ndimage
from tqdm import tqdm 
import pandas as pd

from skimage.morphology import ball, disk, dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing
from skimage.measure import label,regionprops, perimeter
from skimage.morphology import binary_dilation, binary_opening
from skimage.filters import roberts, sobel
from skimage import measure, feature
from skimage.segmentation import clear_border
from skimage import data
from scipy import ndimage
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import dicom
import scipy.misc
import numpy as np




import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from numba import autojit
import zarr
from PIL import Image
import cv2
import scipy.spatial.distance as dist
import gc
import warnings
from solo.unet_utils import *
from solo.unet_models import *
#from solo.preproc_utils import *
#from solo.tianchi import *

import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from keras import backend as K
K.set_image_dim_ordering('th') 


#from inception_unet import *
np.random.seed(1337)



os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"


warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [3]:
def predict_on_scan(scan):
    model = unet_model()
    model.load_weights(luna_path + 'Data/model/Unet_fulldatagen.h5')

    num_test = scan.shape[0]
    scan = scan.reshape(num_test, 1, 512, 512)
    imgs_mask_test = np.ndarray([num_test, 1, 512, 512],dtype=np.float32)
    for i in range(num_test):
        imgs_mask_test[i] = model.predict([scan[i:i+1]], verbose=0)[0]
    return imgs_mask_test

# 数据验证

In [ ]:
imgs_test = np.load("/Volumes/solo/ali/output/images_0000_0065.npy").astype(np.float32)
imgs_mask_test_true = np.load("/Volumes/solo/ali/output/masks_0000_0065.npy").astype(np.float32)

In [ ]:
pred = predict_on_scan(imgs_test)

In [ ]:
imgs_test.shape

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
f, plots = plt.subplots(4, 4, sharex='all', sharey='all', figsize=(20, 20))

for i in range(16):
    plots[i // 4, i % 4].axis('off')
    plots[i // 4, i % 4].imshow(np.squeeze(imgs_test)[i+4])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
f, plots = plt.subplots(4, 4, sharex='all', sharey='all', figsize=(20, 20))

for i in range(16):
    plots[i // 4, i % 4].axis('off')
    plots[i // 4, i % 4].imshow(np.squeeze(imgs_mask_test_true)[i+4])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
f, plots = plt.subplots(4, 4, sharex='all', sharey='all', figsize=(20, 20))

for i in range(16):
    plots[i // 4, i % 4].axis('off')
    plots[i // 4, i % 4].imshow(np.squeeze(pred)[i+4])

# 处理mask

In [ ]:
temp = np.squeeze(pred)
temp.shape

In [ ]:

f, plots = plt.subplots(4, 4, sharex='all', sharey='all', figsize=(20, 20))

for i in range(16):
    plots[i // 4, i % 4].axis('off')
    plots[i // 4, i % 4].imshow(temp[i+4])

In [ ]:

temp = skimage.morphology.binary_opening(temp,np.ones([5,5,5]))

labels= measure.label(temp)



In [ ]:
labels_true = measure.label(np.squeeze(imgs_mask_test_true))

In [ ]:
total = []
props = regionprops(labels)
for i in range(len(props)):
    total.append([props[i]['Centroid'][0],props[i]['Centroid'][1],props[i]['Centroid'][2],props[i]['EquivDiameter']])

In [ ]:
total1 = []
props1 = regionprops(labels_true)
for i in range(len(props1)):
    total1.append([props1[i]['Centroid'][0],props1[i]['Centroid'][1],props1[i]['Centroid'][2],props1[i]['EquivDiameter']])
    

In [ ]:
len(total1),total1,len(total),total

In [ ]:
f, plots = plt.subplots(4, 4, sharex='all', sharey='all', figsize=(20, 20))

for i in range(16):
    plots[i // 4, i % 4].axis('off')
    plots[i // 4, i % 4].imshow(labels[i+4])